In [64]:
import pandas as pd
import numpy as np
import itertools as it
import networkx as nx
# Documentacion de la libreria: http://networkx.readthedocs.io/en/networkx-1.11/

import math

In [65]:
def filterData(df, isTraining, date):
    """
        Funcion que devuelve el conjunto de problemas que tienen status AC o PE
        Si isTraining es true, entonces la funcion sacara el training_set, si no, sacara el evaluation_set
        date es la fecha de particion
    """
    
    if isTraining:
        df = df[df['submissionDate'] < date]
        df = df.loc[df['status'].isin(['AC', 'PE'])]
    else:
        df = df[df['submissionDate'] >= date]
    
    

    return df

In [66]:
def compareNodes(f_list, s_list):
    """
        Funcion que devuelve el numero de usuarios que han hecho ambos problemas
    """
    peso = len(np.intersect1d(f_list, s_list))
    
    return peso
    
def createLinks(prob_us_set, nodos):
    """
        Funcion que crea los enlaces del grafo a partir de la informacion contenida en el conjunto que se le
        pasa a la funcion
    """
    resultado = list() 
    
    # hago todas las posibles combinaciones de problemas
    for fst, snd in it.combinations(nodos, 2):
        # obtengo el peso pasando la lista de usuarios que ha hecho cada problema
        peso = compareNodes(prob_us_set[fst], prob_us_set[snd])
        if peso >= 1:
            resultado.append((fst, snd, peso))
            
            
            
    return resultado

In [67]:
def filterWeight(weightUmbral, linksToFilter):
    """
        Funcion que filtra los enlaces de un grafo, para que el peso sea mayor o igual al dado
    """
    
    result = [(x, y, z) for (x, y, z) in linksToFilter if z >= weightUmbral]
    
    return result
    

In [68]:
def create_graph_nx(list_nodes, list_links):
    """
        Funcion que crea un grafo de tipo Graph de la libreria NetworkX
        Construccion del grafo: http://networkx.readthedocs.io/en/networkx-1.11/tutorial/tutorial.html#what-to-use-as-nodes-and-edges
    """
    grafo = nx.Graph() # creo la variable grafo

    # incluyo los nodos del grafo 
    grafo.add_nodes_from(list_nodes)

    # se incluyen las tuplas de enlaces con el peso del enlace
    # es una lista de la forma [(Nodo1, Nodo2, peso), ......]
    grafo.add_weighted_edges_from(list_links)

    return grafo

In [69]:
# MAIN
# ---------

# se guarda en la variable df (DataFrame) toda la base de datos
df = pd.read_csv('bbdd_orderbydate.csv')

# aqui quito los problemas que no existian despues de la fecha umbral
df = df[df['problem_id'] <= 511] 

# construyo el conjunto de entrenamiento
training_set = filterData(df, True, "2016-10-21 00:00:00")

print(len(training_set))

# obtengo los nodos del grafo:
nodes = training_set.problem_id.unique()


# creo un diccionario que va a tener a los problemas como keys y los valores seran los
# usuarios que han hecho ese problema
grouped = training_set.groupby('problem_id')['user_id'].apply(list)

print(len(nodes))
# print(grouped)


10262
169


In [70]:
print(training_set)

       problem_id  user_id status       submissionDate
0              10        5     AC  2014-02-17 15:27:07
1               2        6     AC  2014-02-17 15:39:17
2               2        9     AC  2014-02-18 00:30:14
3              10        9     AC  2014-02-18 00:34:46
4               4        9     AC  2014-02-18 00:50:28
5               6        9     AC  2014-02-18 00:52:11
6              13        9     AC  2014-02-18 00:53:40
9              15        8     AC  2014-02-19 19:58:03
10              4        8     AC  2014-02-20 14:23:30
11             39       16     AC  2014-02-20 15:44:33
12             39       17     AC  2014-02-20 16:53:34
13             13       12     AC  2014-02-21 11:08:38
15             33       12     AC  2014-02-21 11:58:32
16             39       12     AC  2014-02-21 12:05:49
17             44       12     AC  2014-02-21 12:52:44
18             44       15     AC  2014-02-21 14:51:52
19             44       22     AC  2014-02-21 14:58:55
21        

In [71]:
# OBTENCION DEL EVALUATION_SET
# -------

# ahota saco el evaluation_set
evaluation_set = filterData(df, False, "2016-10-21 00:00:00")

print(evaluation_set)

# creo un diccionario que va a tener a los usuarios como keys y a los problemas que ha hecho como valores
# a partir del conjunto de entrenamiento
grouped_user_eval = evaluation_set.groupby('user_id')['problem_id'].apply(list)

# convierto la serie en un dataframe
df_users_eval = pd.DataFrame({'user_id':grouped_user_eval.index, 'list_problem_id':grouped_user_eval.values})

print(df_users_eval)

       problem_id  user_id status       submissionDate
16939         469      799     RF  2016-10-21 00:08:23
16940         325     3832     AC  2016-10-21 00:08:35
16942         469     3757     AC  2016-10-21 00:41:36
16943         469     3792    RTE  2016-10-21 01:04:08
16944         469      810     AC  2016-10-21 01:48:35
16945         438     3943     AC  2016-10-21 02:23:56
16946         469     3773     AC  2016-10-21 05:00:58
16947         346     3728     AC  2016-10-21 09:06:38
16948         469     3736     CE  2016-10-21 09:38:14
16949         438     3785     TL  2016-10-21 09:43:52
16950         469     3704     AC  2016-10-21 10:04:03
16951         469     2912     AC  2016-10-21 10:20:36
16952         213     3136     AC  2016-10-21 10:28:23
16953         213     3088     AC  2016-10-21 10:29:26
16954         469     3708    RTE  2016-10-21 10:35:15
16956         235     3136    RTE  2016-10-21 10:37:58
16957         183     3880     AC  2016-10-21 11:38:11
16958     

In [72]:

# creo los enlaces a partir de la informacion de los nodos
links = createLinks(grouped, nodes)
# ahora filtro el grafo para que los enlaces solo tengan el peso que quiero
linksFiltered = filterWeight(5, links)

print(len(linksFiltered))

# aqui creo el grafo 
graph = create_graph_nx(nodes, linksFiltered)

10837


In [73]:
def apply_waa(row, graph):
    """
        Funcion que devuelve para cada par de nodos, el valor de weighted adar adamic
    """
    
    # primero tengo que calcular los common neighbors de ambos items
    cn_list = nx.common_neighbors(graph, row['one'], row['two'])
    
    # ahora tengo que hacer el sumatorio del valor para cada elemento de cn_list
    value_waa = sum([((graph[row['one']][x]['weight'] + graph[row['two']][x]['weight']) / math.log(1 + graph.degree(x, weight="weight"), 10) )  for x in cn_list])    
    
    return value_waa

def create_waa_data(graph, nodes):

    # Ahora voy a construir un DataFrame que tenga dos columnas con todas las posibles combinaciones de problemas, y otra 
    # columna con el valor de waa para ese par de problemas
    fst_column = list()
    snd_column = list()
    for fst, snd in it.combinations(nodes, 2):
        fst_column.append(fst)
        snd_column.append(snd)

    d = {'one' : fst_column,
        'two' : snd_column}
    dataFrame_waa = pd.DataFrame(d)


    # Aplico la funcion a cada fila
    dataFrame_waa['waa'] = dataFrame_waa.apply (lambda row: apply_waa(row, graph), axis=1)


    return dataFrame_waa


In [74]:
waa_df = create_waa_data(graph, nodes)
print(waa_df)

       one  two          waa
0       10    2  1462.474284
1       10    4  1009.032036
2       10    6  1048.521115
3       10   13  1147.958613
4       10   15  1024.765951
5       10   39  1830.684676
6       10   33  1448.647842
7       10   44  1518.935641
8       10   19   802.325156
9       10   27   929.599299
10      10   60   742.604183
11      10  100  1079.106633
12      10   93   292.697572
13      10   53   866.090838
14      10   51  1068.853518
15      10   49  1348.293835
16      10   70  1103.916438
17      10   62  1164.420923
18      10   35   941.781187
19      10   81   697.559404
20      10    8   761.375553
21      10   17   630.851657
22      10   23   827.728180
23      10   25     0.000000
24      10   86   904.418707
25      10   29   924.258161
26      10   31     0.000000
27      10   47     0.000000
28      10   65   793.047198
29      10   73   847.812158
...    ...  ...          ...
14166  510  509   994.572613
14167  510  507   681.123551
14168  511  50

In [75]:
# creo un diccionario que va a tener a los usuarios como keys y a los problemas que ha hecho como valores
# a partir del conjunto de entrenamiento
grouped_user = training_set.groupby('user_id')['problem_id'].apply(list)

# convierto la serie en un dataframe
df_users = pd.DataFrame({'user_id':grouped_user.index, 'list_problem_id':grouped_user.values})

print(df_users)

                                        list_problem_id  user_id
0                                     [10, 76, 489, 39]        5
1                                              [2, 487]        6
2                      [39, 44, 76, 105, 109, 114, 117]        7
3     [15, 4, 19, 27, 60, 53, 105, 76, 83, 90, 147, ...        8
4          [2, 10, 4, 6, 13, 253, 57, 62, 143, 83, 109]        9
5                                             [307, 39]       10
6                           [33, 109, 39, 93, 100, 155]       11
7     [13, 33, 39, 44, 100, 51, 49, 70, 81, 53, 130,...       12
8                                         [44, 187, 39]       15
9                                                  [39]       16
10                                             [39, 86]       17
11                            [119, 114, 316, 507, 506]       18
12                                                 [44]       22
13                                            [100, 13]       24
14    [258, 141, 309, 310

In [76]:
def lenProblemsDone(row, set_filter):
    """
        Funcion auxiliar que calcula cuanto problemas ha hecho cada usuario en un conjunto: training o evaluation
    """
    # saco el dataframe que contendra solo una fila con la lista de problemas que ha hecho el usuario
    df_filter = set_filter[set_filter['user_id'] == row['user_id']]
    
    if df_filter.empty:
        # si esta vacio, entonces es que el usuario no ha hecho problemas en ese conjunto
        return 0
    else:
        # sino, devuelvo la longitud de la lista de problemas
        return len(df_filter['list_problem_id'].iloc[0]) 
    
    

In [77]:
# aqui voy a hacer el filtro de usuarios de forma que para hacer las recomendaciones solo tengamos en 
# cuenta aquellos usuarios que han hecho 5 o mas problemas tanto antes de la fecha limite como despues

# primero guardo la lista de usuarios
user_list = df.user_id.unique()

# la meto en un dataframe 
column_user_filter = {'user_id': user_list}
datraframe_user_filter = pd.DataFrame.from_dict(column_user_filter)


# ahora tengo que calcular para cada fila, el numero de problemas que han hecho en el training_set, evaluation_set
datraframe_user_filter['len_training'] = datraframe_user_filter.apply (lambda row: lenProblemsDone(row, df_users), axis=1)
datraframe_user_filter['len_evaluation'] = datraframe_user_filter.apply (lambda row: lenProblemsDone(row, df_users_eval), axis=1)
print(datraframe_user_filter)


      user_id  len_training  len_evaluation
0           5             4               1
1           6             2               0
2           9            11               3
3           8            47               0
4          16             1               0
5          17             2               2
6          12            12               0
7          18             5               8
8          15             3               1
9          22             1               0
10          7             7               0
11         27             0               0
12         28             1               0
13         29             1               0
14         32             2               0
15         35            41               1
16         33             9               0
17         39            11               0
18         31             9               0
19         40             1               0
20         44             8               0
21         48             0     

In [78]:
# ahora tengo que hacer el filtro en este dataframe, de forma que solo aparezcan las filas en las que len_training y 
# len_evaluation sea >=5
datraframe_user_filter = datraframe_user_filter[(datraframe_user_filter['len_training'] >= 5) & (datraframe_user_filter['len_evaluation'] >=5)]
print(datraframe_user_filter)

# aqui voy a guardar la lista de usuarios a los que voy a recomendar
user_list_to_recommend = sorted(datraframe_user_filter['user_id'].tolist())
print(user_list_to_recommend)
print(len(user_list_to_recommend))


      user_id  len_training  len_evaluation
7          18             5               8
29         60            18              14
118        25            22               7
261       130            80               6
299       414            49              61
316       443            53               8
317       448            43              21
483       689            33               7
619       912            13              11
633       935           127               9
1196     1711           128              16
1288     1893             7               6
1311     1952            19               9
1339     1955             9               5
1416     2038             5               5
1423     2096            12              13
1446     2051            11               8
1447     2025            12               8
1448     2120             7               8
1451     2041             9               7
1528     2257             6              13
1694     2576             5     

In [79]:
# ahora tengo que filtrar df_users para que solo contenga las filas en las que los usuarios
# pertenecen a la anterior lista

df_users = df_users[df_users['user_id'].isin(user_list_to_recommend)]
print(df_users)

                                        list_problem_id  user_id
11                            [119, 114, 316, 507, 506]       18
14    [258, 141, 309, 310, 390, 187, 510, 438, 13, 2...       25
39    [100, 62, 2, 57, 53, 23, 134, 15, 147, 233, 24...       60
90    [309, 150, 183, 237, 191, 187, 39, 209, 70, 15...      130
241   [213, 2, 109, 114, 10, 436, 438, 437, 404, 239...      414
255   [39, 100, 247, 150, 183, 44, 471, 109, 307, 30...      443
257   [255, 257, 314, 311, 315, 310, 282, 243, 275, ...      448
392   [171, 272, 282, 209, 373, 155, 2, 39, 134, 471...      689
512   [241, 269, 2, 340, 65, 379, 316, 275, 309, 259...      912
523   [65, 241, 239, 105, 231, 269, 275, 331, 257, 1...      935
933   [507, 33, 83, 217, 312, 256, 49, 4, 503, 254, ...     1711
997                      [39, 49, 465, 159, 134, 13, 2]     1893
1021  [325, 441, 237, 183, 438, 510, 226, 4, 327, 21...     1952
1024      [325, 327, 441, 468, 471, 503, 226, 109, 446]     1955
1066  [155, 505, 49, 162,

In [80]:
# creo el nuevo dataframe con los resultados 
column_problem_recomend = {'problem_id': nodes}
dataframe_problem_recomend = pd.DataFrame.from_dict(column_problem_recomend)

print(dataframe_problem_recomend)

     problem_id
0            10
1             2
2             4
3             6
4            13
5            15
6            39
7            33
8            44
9            19
10           27
11           60
12          100
13           93
14           53
15           51
16           49
17           70
18           62
19           35
20           81
21            8
22           17
23           23
24           25
25           86
26           29
27           31
28           47
29           65
..          ...
139         444
140         437
141         438
142         443
143         445
144         442
145         440
146         446
147         465
148         471
149         468
150         470
151         469
152         467
153         489
154         485
155         486
156         487
157         484
158         488
159         508
160         510
161         511
162         504
163         503
164         505
165         506
166         502
167         509
168         507

[169 ro

In [81]:
def getSimilarProblems(row, waa_df):
    """
        Funcion que devuelve la lista de los problemas similares del problema de la fila: aquellos que tienen un valor waa
        mayor a cero
    """
    # print(row['user_id'])
    
    # obtengo dos df con los usuarios que tienen usuarios vecinos con el usuario de la fila 
    column_result_one_tmp = waa_df[waa_df['one'] == row['problem_id']]
    column_result_one = column_result_one_tmp[column_result_one_tmp['waa'] > 0]
    column_result_two_tmp = waa_df[waa_df['two'] == row['problem_id']]
    column_result_two = column_result_two_tmp[column_result_two_tmp['waa'] > 0]
    
    # saco las listas de usuarios con usuarios comunes
    list_one = list(column_result_one['two'])
    list_two = list(column_result_two['one'])
    
    # la concateno sin tener en cuenta repeticiones, porque nunca va a haber
    list_waa = list_one + list_two
    
    # print(list_waa)
    
    if list_waa == []: # sino tiene vecinos en comun, pongo toda la lista de nodos
        list_waa = graph.nodes()
        list_waa.remove(row['problem_id']) # y elimino el nodo que estoy mirando
    
    # hago el filtro de los k mejores
    return list_waa

In [82]:
dataframe_problem_recomend['neighbors'] = dataframe_problem_recomend.apply (lambda row: getSimilarProblems(row, waa_df), axis=1)

# aqui tengo la lista de usuarios con sus k usuarios similares
print(dataframe_problem_recomend)

     problem_id                                          neighbors
0            10  [2, 4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100,...
1             2  [4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 93...
2             4  [6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 93, 5...
3             6  [13, 15, 39, 33, 44, 19, 27, 60, 100, 93, 53, ...
4            13  [15, 39, 33, 44, 19, 27, 60, 100, 93, 53, 51, ...
5            15  [39, 33, 44, 19, 27, 60, 100, 93, 53, 51, 49, ...
6            39  [33, 44, 19, 27, 60, 100, 93, 53, 51, 49, 70, ...
7            33  [44, 19, 27, 60, 100, 93, 53, 51, 49, 70, 62, ...
8            44  [19, 27, 60, 100, 93, 53, 51, 49, 70, 62, 35, ...
9            19  [27, 60, 100, 93, 53, 51, 49, 70, 62, 35, 81, ...
10           27  [60, 100, 93, 53, 51, 49, 70, 62, 35, 81, 8, 1...
11           60  [100, 93, 53, 51, 49, 70, 62, 35, 81, 8, 17, 2...
12          100  [93, 53, 51, 49, 70, 62, 35, 81, 8, 17, 23, 86...
13           93  [53, 51, 49, 70, 62, 35, 81, 8, 17, 23, 86, 2

In [83]:
print(grouped)
# convierto la serie en un dataframe
df_users_recommend = pd.DataFrame({'problem_id':grouped.index, 'list_user_id':grouped.values})
print(df_users_recommend)

problem_id
2      [6, 9, 33, 67, 56, 84, 35, 60, 112, 124, 106, ...
4      [9, 8, 35, 84, 126, 44, 39, 313, 42, 437, 130,...
6      [9, 46, 35, 67, 89, 84, 137, 152, 231, 221, 88...
8      [35, 52, 133, 129, 130, 437, 511, 605, 935, 67...
10     [5, 9, 35, 133, 129, 80, 137, 247, 437, 103, 5...
13     [9, 12, 59, 35, 67, 109, 54, 24, 55, 106, 136,...
15     [8, 35, 67, 55, 133, 129, 84, 167, 205, 231, 2...
17           [35, 133, 129, 437, 103, 206, 62, 89, 1711]
19     [8, 35, 44, 205, 437, 527, 448, 1063, 206, 62,...
23     [35, 67, 60, 437, 206, 62, 103, 864, 1001, 113...
25                                        [35, 103, 130]
27     [8, 35, 67, 167, 39, 440, 448, 437, 130, 488, ...
29     [35, 133, 86, 112, 42, 232, 248, 247, 314, 320...
31                                        [35, 437, 448]
33     [12, 59, 35, 84, 111, 96, 152, 126, 155, 30, 1...
35     [64, 35, 214, 120, 236, 82, 409, 390, 175, 249...
39     [16, 17, 12, 7, 31, 40, 33, 66, 68, 71, 69, 70...
44     [12, 15, 22, 

In [84]:
def waa_value(one, two, graph):
    """
        Funcion que devuelve para cada par de nodos, el valor de weighted adar adamic
    """
    
    # primero tengo que calcular los common neighbors de ambos items
    cn_list = nx.common_neighbors(graph, one, two)
    
    # ahora tengo que hacer el sumatorio del valor para cada elemento de cn_list
    value_waa = sum([((graph[one][x]['weight'] + graph[two][x]['weight']) / math.log(1 + graph.degree(x, weight="weight"), 10)) for x in cn_list])    
    
    
    return value_waa

In [85]:
def getWeights(row, graph):
    """
        Funcion que calcula la suma de todos los valores de waa de todos los nodos con los que tiene waa
    """
    
    # primero obtengo la lista de los vecinos
    neighbors_list = row['neighbors']
    
    # print(neighbors_list)
    
    problem = row['problem_id']
    suma = 0
    
    for elem in neighbors_list:
        # print(suma)
        # print(graph[user][elem]['weight'])
        suma = suma + waa_value(problem, elem, graph)
        
    return suma

In [86]:
def getPonderaciones(row, graph):
    """
        Funcion que calcula la ponderacion para cada problema vecino del de la fila
        Calculo la ponderacion diviendo el peso del enlace que enlaza cada problema con problem_id con la suma 
        total de los pesos
    """
    # primero obtengo la lista de los vecinos
    neighbors_list = row['neighbors']
    
    # obtengo la suma de pesos de los enlaces de esa lista
    total_weight = row['total_weight']
    
    # obtengo el id del usuario al que quiero recomendar
    problem = row['problem_id']
    
    lista_ponderaciones = list()
    
    for elem in neighbors_list:
        
        # obtengo el peso del enlace
        peso_enlace = waa_value(problem, elem, graph)
        
        if total_weight == 0:
            ponderacion = 0
        else:
            # hago la ponderacion
            ponderacion = peso_enlace/total_weight
        
        lista_ponderaciones.append(ponderacion)
        
    return lista_ponderaciones
    

In [87]:
# ahora voy a incluir una nueva columna que tenga las ponderaciones (suma de todos los enlaces) de cada usuario
dataframe_problem_recomend['total_weight'] = dataframe_problem_recomend.apply (lambda row: getWeights(row, graph), axis=1)

print(dataframe_problem_recomend)

     problem_id                                          neighbors  \
0            10  [2, 4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100,...   
1             2  [4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 93...   
2             4  [6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 93, 5...   
3             6  [13, 15, 39, 33, 44, 19, 27, 60, 100, 93, 53, ...   
4            13  [15, 39, 33, 44, 19, 27, 60, 100, 93, 53, 51, ...   
5            15  [39, 33, 44, 19, 27, 60, 100, 93, 53, 51, 49, ...   
6            39  [33, 44, 19, 27, 60, 100, 93, 53, 51, 49, 70, ...   
7            33  [44, 19, 27, 60, 100, 93, 53, 51, 49, 70, 62, ...   
8            44  [19, 27, 60, 100, 93, 53, 51, 49, 70, 62, 35, ...   
9            19  [27, 60, 100, 93, 53, 51, 49, 70, 62, 35, 81, ...   
10           27  [60, 100, 93, 53, 51, 49, 70, 62, 35, 81, 8, 1...   
11           60  [100, 93, 53, 51, 49, 70, 62, 35, 81, 8, 17, 2...   
12          100  [93, 53, 51, 49, 70, 62, 35, 81, 8, 17, 23, 86...   
13           93  [53

In [88]:
# para cada usuario a "recomendar" para cada problema miro el numero de apariciones en su lista

dataframe_problem_recomend['score'] = dataframe_problem_recomend.apply (lambda row: getPonderaciones(row, graph), axis=1)

print(dataframe_problem_recomend)

     problem_id                                          neighbors  \
0            10  [2, 4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100,...   
1             2  [4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 93...   
2             4  [6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 93, 5...   
3             6  [13, 15, 39, 33, 44, 19, 27, 60, 100, 93, 53, ...   
4            13  [15, 39, 33, 44, 19, 27, 60, 100, 93, 53, 51, ...   
5            15  [39, 33, 44, 19, 27, 60, 100, 93, 53, 51, 49, ...   
6            39  [33, 44, 19, 27, 60, 100, 93, 53, 51, 49, 70, ...   
7            33  [44, 19, 27, 60, 100, 93, 53, 51, 49, 70, 62, ...   
8            44  [19, 27, 60, 100, 93, 53, 51, 49, 70, 62, 35, ...   
9            19  [27, 60, 100, 93, 53, 51, 49, 70, 62, 35, 81, ...   
10           27  [60, 100, 93, 53, 51, 49, 70, 62, 35, 81, 8, 1...   
11           60  [100, 93, 53, 51, 49, 70, 62, 35, 81, 8, 17, 2...   
12          100  [93, 53, 51, 49, 70, 62, 35, 81, 8, 17, 23, 86...   
13           93  [53

In [89]:
def getUsersFromSimilarProblems(row, df_users_recommend):
    """
        Funcion que va a devolver por cada fila una lista procedente de la concatenacion de listas de usuarios que han
        realizado los problemas similares al de la fila. Además eliminara los usuarios que ya hayan realizado el problema.
    """
    
    # obtengo la lista de usuarios que han hecho el problema en cuestion
    list_problems_users = df_users_recommend[df_users_recommend['problem_id'] == row['problem_id']]
    list_problems_user = list(list_problems_users['list_user_id'])[0]
   
    # print(list_problems_user)

    # lista resultante de la concatenacion de las listas de usuarios de los problemas similares
    list_result = list()
    
    # obtengo la longitud de la lista de vecinos de ese usuario
    list_neighbors = row['neighbors']
    k = len(list_neighbors)
    
    # recorro la lista de usuarios vecinos 
    for i in range(0, k):
        # print(row['list_similar_users'][i])
        # aqui saco la lista de usuarios que han hecho el problema similar
        list_problems_df = df_users_recommend[df_users_recommend['problem_id'] == row['neighbors'][i]]
        lista_problemas_comprobar = list(list_problems_df['list_user_id'])[0]
        
        # print("----------")
        #print(lista_problemas_comprobar)
        
        # aqui hago el filtro para que no se incluyan los usuarios que ya ha hechon el problema
        list_users = [x for x in lista_problemas_comprobar if x not in list_problems_user]
        
        # ahora concateno el resultado
        list_result = list_result + list_users
        # print(list_problems)
        # print(list_result)
        # print("---------------")
    
    return list_result

In [90]:
# ahora para cada problema, hacer una lista de los usuarios similares a los que han realizado ese problema, 
# que no sean usuarios que han realizado ya ese problema
dataframe_problem_recomend['list_users'] = dataframe_problem_recomend.apply (lambda row: getUsersFromSimilarProblems(row, df_users_recommend), axis=1)
print(dataframe_problem_recomend)

     problem_id                                          neighbors  \
0            10  [2, 4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100,...   
1             2  [4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 93...   
2             4  [6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 93, 5...   
3             6  [13, 15, 39, 33, 44, 19, 27, 60, 100, 93, 53, ...   
4            13  [15, 39, 33, 44, 19, 27, 60, 100, 93, 53, 51, ...   
5            15  [39, 33, 44, 19, 27, 60, 100, 93, 53, 51, 49, ...   
6            39  [33, 44, 19, 27, 60, 100, 93, 53, 51, 49, 70, ...   
7            33  [44, 19, 27, 60, 100, 93, 53, 51, 49, 70, 62, ...   
8            44  [19, 27, 60, 100, 93, 53, 51, 49, 70, 62, 35, ...   
9            19  [27, 60, 100, 93, 53, 51, 49, 70, 62, 35, 81, ...   
10           27  [60, 100, 93, 53, 51, 49, 70, 62, 35, 81, 8, 1...   
11           60  [100, 93, 53, 51, 49, 70, 62, 35, 81, 8, 17, 2...   
12          100  [93, 53, 51, 49, 70, 62, 35, 81, 8, 17, 23, 86...   
13           93  [53

In [91]:
def delRepetitions(row):
    """
        Funcion auxiliar para evitar que salgan repeticiones en las recomendaciones. Saco la lista de posibles 
        recomendaciones con valores unicos
    """
    conjunto_vacio = set()
    
    # esto sirve para que se haga mas rapido la comprobacion de si el elemento esta en la lista o no
    function_add = conjunto_vacio.add
    
    # hago la lista intensional, para mantener el orden dado en la lista original
    return [x for x in row['list_users'] if not (x in conjunto_vacio or function_add(x))]

In [92]:

# tengo que contar el numero de apariciones de cada usuario en la lista de usuarios

# voy a sacar primero una lista sin repeticiones
# ahora voy a crear una nueva columna que contenga la lista de usuarios sin repeticiones
dataframe_problem_recomend['lista_users_unique'] = dataframe_problem_recomend.apply(lambda row: delRepetitions(row), axis=1)

print(dataframe_problem_recomend)

     problem_id                                          neighbors  \
0            10  [2, 4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100,...   
1             2  [4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 93...   
2             4  [6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 93, 5...   
3             6  [13, 15, 39, 33, 44, 19, 27, 60, 100, 93, 53, ...   
4            13  [15, 39, 33, 44, 19, 27, 60, 100, 93, 53, 51, ...   
5            15  [39, 33, 44, 19, 27, 60, 100, 93, 53, 51, 49, ...   
6            39  [33, 44, 19, 27, 60, 100, 93, 53, 51, 49, 70, ...   
7            33  [44, 19, 27, 60, 100, 93, 53, 51, 49, 70, 62, ...   
8            44  [19, 27, 60, 100, 93, 53, 51, 49, 70, 62, 35, ...   
9            19  [27, 60, 100, 93, 53, 51, 49, 70, 62, 35, 81, ...   
10           27  [60, 100, 93, 53, 51, 49, 70, 62, 35, 81, 8, 1...   
11           60  [100, 93, 53, 51, 49, 70, 62, 35, 81, 8, 17, 2...   
12          100  [93, 53, 51, 49, 70, 62, 35, 81, 8, 17, 23, 86...   
13           93  [53

In [93]:
# elimino la columna de la lista de problemas (la que tiene repeticiones), ya que no interesa
del dataframe_problem_recomend['list_users']
# elimino la columna de total_weight, ya que no interesa
del dataframe_problem_recomend['total_weight']

print(dataframe_problem_recomend)

     problem_id                                          neighbors  \
0            10  [2, 4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100,...   
1             2  [4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 93...   
2             4  [6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 93, 5...   
3             6  [13, 15, 39, 33, 44, 19, 27, 60, 100, 93, 53, ...   
4            13  [15, 39, 33, 44, 19, 27, 60, 100, 93, 53, 51, ...   
5            15  [39, 33, 44, 19, 27, 60, 100, 93, 53, 51, 49, ...   
6            39  [33, 44, 19, 27, 60, 100, 93, 53, 51, 49, 70, ...   
7            33  [44, 19, 27, 60, 100, 93, 53, 51, 49, 70, 62, ...   
8            44  [19, 27, 60, 100, 93, 53, 51, 49, 70, 62, 35, ...   
9            19  [27, 60, 100, 93, 53, 51, 49, 70, 62, 35, 81, ...   
10           27  [60, 100, 93, 53, 51, 49, 70, 62, 35, 81, 8, 1...   
11           60  [100, 93, 53, 51, 49, 70, 62, 35, 81, 8, 17, 2...   
12          100  [93, 53, 51, 49, 70, 62, 35, 81, 8, 17, 23, 86...   
13           93  [53

In [94]:
def getListUsersFromSimilarProblems(row, df_users_recommend):
    """
        Funcion que va a devolver por cada fila una lista procedente de listas la concatenacion de listas de usuarios que han
        hecho los problemas similares a ese. Además eliminara los usuarios que ya hayan hecho el problema
    """
    
    # obtengo la lista de problemas que ha hecho el usuario en cuestion
    list_problems_users = df_users_recommend[df_users_recommend['problem_id'] == row['problem_id']]
    list_problems_user = list(list_problems_users['list_user_id'])[0]
   
    # lista resultante 
    list_result = list(list())
    
    # obtengo la longitud de la lista de vecinos de ese problema
    list_neighbors = row['neighbors']
    k = len(list_neighbors)
    
    # recorro la lista de problemas vecinos 
    for i in range(0, k):
        # print(row['list_similar_users'][i])
        # aqui saco la lista de problemas que ha hecho el usuario similar
        list_users_df = df_users_recommend[df_users_recommend['problem_id'] == row['neighbors'][i]]
        lista_problemas_comprobar = list(list_users_df['list_user_id'])[0]
        
        # aqui hago el filtro para que no se incluyan los problemas que ya ha hecho el usuario
        list_problems = [x for x in lista_problemas_comprobar if x not in list_problems_user]
        
        # ahora incluyo la lista en la lista
        list_result.append(list_problems)
        # print(list_problems)
        # print(list_result)
        # print("---------------")
    
    return list_result

In [95]:
dataframe_problem_recomend['lista_users_por_problema'] = dataframe_problem_recomend.apply(lambda row: getListUsersFromSimilarProblems(row, df_users_recommend), axis=1)

print(dataframe_problem_recomend)


     problem_id                                          neighbors  \
0            10  [2, 4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100,...   
1             2  [4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 93...   
2             4  [6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 93, 5...   
3             6  [13, 15, 39, 33, 44, 19, 27, 60, 100, 93, 53, ...   
4            13  [15, 39, 33, 44, 19, 27, 60, 100, 93, 53, 51, ...   
5            15  [39, 33, 44, 19, 27, 60, 100, 93, 53, 51, 49, ...   
6            39  [33, 44, 19, 27, 60, 100, 93, 53, 51, 49, 70, ...   
7            33  [44, 19, 27, 60, 100, 93, 53, 51, 49, 70, 62, ...   
8            44  [19, 27, 60, 100, 93, 53, 51, 49, 70, 62, 35, ...   
9            19  [27, 60, 100, 93, 53, 51, 49, 70, 62, 35, 81, ...   
10           27  [60, 100, 93, 53, 51, 49, 70, 62, 35, 81, 8, 1...   
11           60  [100, 93, 53, 51, 49, 70, 62, 35, 81, 8, 17, 2...   
12          100  [93, 53, 51, 49, 70, 62, 35, 81, 8, 17, 23, 86...   
13           93  [53

In [96]:
# ahora voy a separar cada problema_user para hacer la cuenta
# creo un nuevo dataframe que agrupa por el problema con su posible usuario
df_separation = dataframe_problem_recomend.groupby(['problem_id']).lista_users_unique.apply(lambda x: pd.DataFrame(x.values[0])).reset_index().drop('level_1', axis = 1)

df_separation.columns = ['problem_id', 'possible_user', ]

print(df_separation)

        problem_id  possible_user
0                2            126
1                2             39
2                2            313
3                2             42
4                2           1902
5                2           1983
6                2           1959
7                2           1988
8                2           1952
9                2           1962
10               2           3497
11               2             46
12               2            152
13               2            221
14               2             88
15               2            194
16               2             90
17               2             99
18               2            155
19               2            542
20               2            896
21               2           1186
22               2            598
23               2            584
24               2            596
25               2            594
26               2            593
27               2            581
28            

In [97]:
def getScoring(row, dataframe_problem_recomend):
    """
        Funcion en la que por cada usuario-problema, hace el scoring para ese problema (para ese usuario)
        sumando todas las ponderaciones de ese problema para ese usuario
    """
     # obtengo la lista de listas de usuarios que han hecho los vecinos del problema
    list_problems_users_df = dataframe_problem_recomend[dataframe_problem_recomend['problem_id'] == row['problem_id']]
    list_user_per_problem = list(list_problems_users_df['lista_users_por_problema'])[0]
    
    # saco tambien la lista de scores por vecinos
    list_scores_per_problem = list(list_problems_users_df['score'])[0]
    
    # print(list_scores_per_user)
    
    # obtengo el usuario del que quiero calcular el scoring
    user = row['possible_user']
    # print(problem)
    
    suma = 0
    k = len(list_user_per_problem)
    
    # por cada lista de la lista, saco la ponderacion de su correspondiente usuario (el usuario que lo ha hecho)
    # si problem aparece en la lista, sumo esa ponderacion
    for i in range(0, k):
        lista_a_comprobar = list_user_per_problem[i]
        # print(lista_a_comprobar)
        if user in lista_a_comprobar:
            # print(list_scores_per_user[i])
            suma = suma + list_scores_per_problem[i]
    
    return suma

In [98]:
# ahora voy a calcular el scoring para cada usuario
df_separation['score'] = df_separation.apply(lambda row: getScoring(row, dataframe_problem_recomend), axis=1)

print(df_separation)

        problem_id  possible_user     score
0                2            126  0.060920
1                2             39  0.082982
2                2            313  0.006899
3                2             42  0.169981
4                2           1902  0.013644
5                2           1983  0.248320
6                2           1959  0.155227
7                2           1988  0.099506
8                2           1952  0.139700
9                2           1962  0.363519
10               2           3497  0.056485
11               2             46  0.060385
12               2            152  0.113815
13               2            221  0.025087
14               2             88  0.016551
15               2            194  0.007036
16               2             90  0.026366
17               2             99  0.027004
18               2            155  0.048806
19               2            542  0.132503
20               2            896  0.070299
21               2           118

In [99]:
# ahora voy a ordenar en funcion del score de mayor a menor para cada usuario
# ahora lo que quiero es ordenar los problemas por cada usuario en funcion de su ponderacion
# primero ordeno por su valor de user y luego por el de ponderacion, de forma que quedan ordenador por su valor de ponderacion
df_separation = df_separation.sort_values(by=['possible_user', 'score'], ascending=False)
print(df_separation)

        problem_id  possible_user     score
223108         322           4018  0.012169
295180         470           4018  0.012064
254072         379           4018  0.011341
211318         311           4018  0.011131
293180         469           4018  0.011006
301118         485           4018  0.010904
248084         365           4018  0.010842
289211         467           4018  0.010814
195777         272           4018  0.010745
191873         266           4018  0.010693
187906         262           4018  0.010666
72509           97           4018  0.010639
305106         487           4018  0.010629
320955         507           4018  0.010620
189887         264           4018  0.010606
87793          124           4018  0.010563
58642           81           4018  0.010536
143130         224           4018  0.010481
252074         374           4018  0.010430
113976         166           4018  0.010428
303113         486           4018  0.010415
244087         355           401

In [100]:
# ahora tengo que hacer un nuevo dataframe con usuario y problema, y una lista de recommendation 

# hago primero la agrupacion por usuario
grouped_r = df_separation.groupby('possible_user')

# hago la agregacion en una lista 
df_recommend = grouped_r.aggregate(lambda x:list(x))

#print(df_recommend)

# vuelvo a crear la estructura de los datos para poder trabajar con ellos
df_recommend = pd.DataFrame({'user_id':df_recommend.index.values, 'problem_id':df_recommend['problem_id'].tolist()})
print(df_recommend)

                                             problem_id  user_id
0     [322, 467, 93, 269, 17, 262, 264, 166, 272, 81...        5
1     [322, 470, 93, 489, 340, 485, 311, 507, 272, 4...        6
2     [322, 489, 467, 365, 93, 262, 269, 17, 272, 26...        7
3     [269, 93, 17, 166, 65, 231, 88, 29, 224, 78, 2...        8
4     [93, 76, 470, 17, 97, 269, 262, 81, 166, 507, ...        9
5     [322, 467, 489, 93, 365, 311, 269, 507, 272, 7...       10
6     [322, 467, 489, 365, 311, 76, 507, 272, 17, 26...       11
7     [322, 467, 93, 365, 76, 17, 97, 311, 266, 507,...       12
8     [322, 467, 365, 93, 311, 272, 485, 489, 97, 50...       15
9     [322, 467, 489, 93, 365, 311, 269, 76, 272, 50...       16
10    [322, 467, 489, 93, 76, 269, 311, 365, 17, 272...       17
11    [489, 322, 39, 33, 109, 327, 2, 254, 44, 162, ...       18
12    [467, 365, 93, 470, 322, 311, 485, 97, 266, 26...       22
13    [322, 93, 365, 76, 17, 358, 97, 81, 222, 88, 2...       24
14    [470, 467, 440, 379

In [101]:
def getProblems(row, df_recommend):
    """
        Funcion que copia los posibles problemas a recomendar para los usuarios a los que tengo que recomendar
    """
    
    # saco la lista de problemas 
    df_lista_problemas_original = df_recommend[df_recommend['user_id'] == row['user_id']]
    lista_problemas_original = list(df_lista_problemas_original['problem_id'])[0]
    
    return lista_problemas_original
    

In [102]:
# df_users # en este dataframe estan los usuarios a los que quiero recomendar

# ahora voy a copiar los problemas para los usuarios a los que tengo que recomendar
df_users['recommendation'] = df_users.apply (lambda row: getProblems(row, df_recommend), axis=1)

print(df_users)

                                        list_problem_id  user_id  \
11                            [119, 114, 316, 507, 506]       18   
14    [258, 141, 309, 310, 390, 187, 510, 438, 13, 2...       25   
39    [100, 62, 2, 57, 53, 23, 134, 15, 147, 233, 24...       60   
90    [309, 150, 183, 237, 191, 187, 39, 209, 70, 15...      130   
241   [213, 2, 109, 114, 10, 436, 438, 437, 404, 239...      414   
255   [39, 100, 247, 150, 183, 44, 471, 109, 307, 30...      443   
257   [255, 257, 314, 311, 315, 310, 282, 243, 275, ...      448   
392   [171, 272, 282, 209, 373, 155, 2, 39, 134, 471...      689   
512   [241, 269, 2, 340, 65, 379, 316, 275, 309, 259...      912   
523   [65, 241, 239, 105, 231, 269, 275, 331, 257, 1...      935   
933   [507, 33, 83, 217, 312, 256, 49, 4, 503, 254, ...     1711   
997                      [39, 49, 465, 159, 134, 13, 2]     1893   
1021  [325, 441, 237, 183, 438, 510, 226, 4, 327, 21...     1952   
1024      [325, 327, 441, 468, 471, 503, 226, 10

In [103]:
# elimino la columna de los problemas, ya que no interesa
del df_users['list_problem_id']

print(df_users)

      user_id                                     recommendation
11         18  [489, 322, 39, 33, 109, 327, 2, 254, 44, 162, ...
14         25  [470, 467, 440, 379, 311, 340, 373, 507, 469, ...
39         60  [322, 467, 470, 93, 311, 76, 97, 507, 17, 262,...
90        130  [467, 93, 365, 17, 76, 81, 266, 322, 507, 262,...
241       414  [467, 93, 470, 311, 365, 76, 17, 97, 507, 266,...
255       443  [467, 470, 322, 311, 365, 507, 340, 272, 469, ...
257       448  [39, 489, 109, 2, 254, 258, 44, 159, 141, 327,...
392       689  [322, 489, 467, 365, 93, 311, 470, 269, 76, 26...
512       912  [358, 322, 440, 470, 489, 282, 279, 373, 39, 5...
523       935  [440, 358, 342, 122, 336, 507, 262, 145, 4, 19...
933      1711  [467, 76, 97, 262, 166, 224, 231, 374, 486, 93...
997      1893  [322, 467, 93, 365, 489, 76, 470, 311, 17, 507...
1021     1952  [467, 365, 470, 311, 485, 507, 266, 272, 487, ...
1024     1955  [379, 467, 470, 311, 365, 469, 507, 485, 272, ...
1066     2025  [467, 365,

In [104]:
def getKrecomFinal(row, k):
    """
        Funcion que saca las k mejores recomendaciones para el usuario
        Lo que hace es coger los primeros k valores de la lista de recomendaciones
    """
    return row['recommendation'][:k]

In [105]:
k = 10

#print(list(range(1, k+1)))

rango = list(range(1, k+1))

for i in rango:
    name_column = 'k_recommendation_' + str(i)
    #print(name_column)
    # ahora saco los k mejores problemas para cada usuario
    df_users[name_column] = df_users.apply(lambda row: getKrecomFinal(row, i), axis=1)

# he incluido una columna para cada cada k recomendacion 
print(df_users)

      user_id                                     recommendation  \
11         18  [489, 322, 39, 33, 109, 327, 2, 254, 44, 162, ...   
14         25  [470, 467, 440, 379, 311, 340, 373, 507, 469, ...   
39         60  [322, 467, 470, 93, 311, 76, 97, 507, 17, 262,...   
90        130  [467, 93, 365, 17, 76, 81, 266, 322, 507, 262,...   
241       414  [467, 93, 470, 311, 365, 76, 17, 97, 507, 266,...   
255       443  [467, 470, 322, 311, 365, 507, 340, 272, 469, ...   
257       448  [39, 489, 109, 2, 254, 258, 44, 159, 141, 327,...   
392       689  [322, 489, 467, 365, 93, 311, 470, 269, 76, 26...   
512       912  [358, 322, 440, 470, 489, 282, 279, 373, 39, 5...   
523       935  [440, 358, 342, 122, 336, 507, 262, 145, 4, 19...   
933      1711  [467, 76, 97, 262, 166, 224, 231, 374, 486, 93...   
997      1893  [322, 467, 93, 365, 489, 76, 470, 311, 17, 507...   
1021     1952  [467, 365, 470, 311, 485, 507, 266, 272, 487, ...   
1024     1955  [379, 467, 470, 311, 365, 469, 50

In [106]:
# elimino las columnas que no me interesan
del df_users['recommendation']

print(df_users)

      user_id k_recommendation_1 k_recommendation_2 k_recommendation_3  \
11         18              [489]         [489, 322]     [489, 322, 39]   
14         25              [470]         [470, 467]    [470, 467, 440]   
39         60              [322]         [322, 467]    [322, 467, 470]   
90        130              [467]          [467, 93]     [467, 93, 365]   
241       414              [467]          [467, 93]     [467, 93, 470]   
255       443              [467]         [467, 470]    [467, 470, 322]   
257       448               [39]          [39, 489]     [39, 489, 109]   
392       689              [322]         [322, 489]    [322, 489, 467]   
512       912              [358]         [358, 322]    [358, 322, 440]   
523       935              [440]         [440, 358]    [440, 358, 342]   
933      1711              [467]          [467, 76]      [467, 76, 97]   
997      1893              [322]         [322, 467]     [322, 467, 93]   
1021     1952              [467]      

In [107]:
# ahora tengo que filtrar df_users_eval para que solo contenga las filas de los usuarios a los que hay que recomendar

df_users_eval_filter = df_users_eval[df_users_eval['user_id'].isin(user_list_to_recommend)]
print(df_users_eval_filter)

                                       list_problem_id  user_id
4               [241, 128, 124, 256, 404, 469, 70, 86]       18
5                    [33, 29, 254, 308, 445, 253, 166]       25
8    [44, 228, 141, 217, 251, 105, 4, 155, 181, 191...       60
13                       [254, 253, 308, 166, 49, 445]      130
22   [39, 393, 390, 249, 233, 25, 27, 29, 35, 53, 5...      414
26             [134, 187, 256, 251, 90, 174, 147, 438]      443
27   [203, 136, 141, 2, 307, 325, 336, 312, 313, 31...      448
34                     [327, 23, 150, 349, 336, 6, 49]      689
44   [128, 247, 44, 81, 272, 279, 282, 322, 331, 37...      912
46          [95, 383, 262, 122, 44, 81, 252, 145, 336]      935
70   [224, 251, 259, 485, 93, 437, 53, 166, 374, 44...     1711
73                       [471, 469, 355, 95, 243, 436]     1893
78         [374, 195, 109, 436, 95, 10, 446, 340, 379]     1952
80                            [44, 213, 136, 162, 243]     1955
90             [136, 253, 187, 342, 203,

In [108]:
list_eval_problems = df_users_eval_filter['list_problem_id'].tolist()


# meto toda la informacion en un dataframe para obtener las metricas
set_df_metric = {'user_id': user_list_to_recommend, 'eval_problems': list_eval_problems}

for i in rango:
    name_column = 'k_recommendation_' + str(i)
    list_recom_problems = list()
    list_recom_problems = df_users[name_column].tolist()
    
    name_column_metric = 'recom_problems_' + str(i)
    set_df_metric[name_column_metric] = list_recom_problems

# he generado un nuevo dataframe con todas las recomendaciones (desde 1 a k) y los problemas realizados en 
# el evaluation set para cada usuario al que quiero recomendar

metric_df = pd.DataFrame.from_dict(set_df_metric)

print(metric_df)

                                        eval_problems recom_problems_1  \
0              [241, 128, 124, 256, 404, 469, 70, 86]            [489]   
1                   [33, 29, 254, 308, 445, 253, 166]            [470]   
2   [44, 228, 141, 217, 251, 105, 4, 155, 181, 191...            [322]   
3                       [254, 253, 308, 166, 49, 445]            [467]   
4   [39, 393, 390, 249, 233, 25, 27, 29, 35, 53, 5...            [467]   
5             [134, 187, 256, 251, 90, 174, 147, 438]            [467]   
6   [203, 136, 141, 2, 307, 325, 336, 312, 313, 31...             [39]   
7                     [327, 23, 150, 349, 336, 6, 49]            [322]   
8   [128, 247, 44, 81, 272, 279, 282, 322, 331, 37...            [358]   
9          [95, 383, 262, 122, 44, 81, 252, 145, 336]            [440]   
10  [224, 251, 259, 485, 93, 437, 53, 166, 374, 44...            [467]   
11                      [471, 469, 355, 95, 243, 436]            [322]   
12        [374, 195, 109, 436, 95, 10,

In [109]:
def one_hit(row, i):
    """
        Funcion que implementa la metrica one hit. Devuelve un 1 si para un usuarios dado, al menos uno
        de los problemas que se le ha recomendado ha sido realizado por ese usuario en el evaluation_set. 
        Cero si no hay ningun problema de los recomendados que haya sido realizado por el usuario
    """
    name_column = 'recom_problems_' + str(i)
    num_problems_common = np.intersect1d(row[name_column], row['eval_problems'])
    
    if len(num_problems_common) >= 1:
        return 1
    else:
        return 0

In [110]:
def mrr(row, i): 
    """
        Funcion que va a implementar la metrica de evaluacion mrr:
        mrr = 1/ranki, donde ranki es la posicion del primer item correcto
    """
    name_column = 'recom_problems_' + str(i)
    num_problems_common = np.intersect1d(row[name_column], row['eval_problems'])
    
    if len(num_problems_common) >= 1:

        # hago la busqueda del primer elemento que esta en la lista de recomendados
        fst_correct_item = -1
        encontrado = False
        i = 0
        while (i < len(row[name_column])) and (encontrado == False):
            if row[name_column][i] in row['eval_problems']:
                # fst_correct_item = row['recom_problems'][i]
                # print(fst_correct_item)
                ranki = i + 1
                encontrado = True
            else:
                i = i + 1
                
        return (1/ranki)

    else:
        return 0

In [111]:
def precision(row, i):
    """
        Funcion que va a implementar la metrica precision en k: 
        (cuantos de los realizados por el usuario estan entre los recomendados) / todos los recomendados
    """
    name_column = 'recom_problems_' + str(i)
    num_problems_common = np.intersect1d(row[name_column], row['eval_problems'])
    
    # print(num_problems_common)
    
    return (len(num_problems_common)/len(row[name_column]))

In [112]:
def recall(row, i):
    """
        Funcion que implementa la metrica recall
        (cuantos de los realizados por el usuario estan entre los recomendados) / todos los evaluados
    """
    name_column = 'recom_problems_' + str(i)
    num_problems_common = np.intersect1d(row[name_column], row['eval_problems'])
    
    # print(num_problems_common)
    
    return (len(num_problems_common)/len(row['eval_problems']))

In [113]:
def f1(row, i):
    """
        Funcion que calcula el f1 en funcion de precision y recall
    """
    name_column_prec = 'precision_' + str(i)
    name_column_rec = 'recall_' + str(i)
    denominador = row[name_column_prec] + row[name_column_rec]
    
    if denominador == 0:
        return 0
    else:
        return (2 * row[name_column_prec] * row[name_column_rec]) / denominador

In [114]:
for i in rango:
    name_one_hit = 'one_hit_' + str(i)
    name_mrr = 'mrr_' + str(i)
    name_precision = 'precision_' + str(i)
    name_recall = 'recall_' + str(i)
    name_f1 = 'f1_' + str(i)
    
    # ahora voy a calcular una metrica para cada usuario
    metric_df[name_one_hit] = metric_df.apply(lambda row: one_hit(row, i), axis=1)
    metric_df[name_mrr] = metric_df.apply(lambda row: mrr(row, i), axis=1)
    metric_df[name_precision] = metric_df.apply(lambda row: precision(row, i), axis=1)
    metric_df[name_recall] = metric_df.apply(lambda row: recall(row, i), axis=1)
    metric_df[name_f1] = metric_df.apply(lambda row: f1(row, i), axis=1)

print(metric_df)

                                        eval_problems recom_problems_1  \
0              [241, 128, 124, 256, 404, 469, 70, 86]            [489]   
1                   [33, 29, 254, 308, 445, 253, 166]            [470]   
2   [44, 228, 141, 217, 251, 105, 4, 155, 181, 191...            [322]   
3                       [254, 253, 308, 166, 49, 445]            [467]   
4   [39, 393, 390, 249, 233, 25, 27, 29, 35, 53, 5...            [467]   
5             [134, 187, 256, 251, 90, 174, 147, 438]            [467]   
6   [203, 136, 141, 2, 307, 325, 336, 312, 313, 31...             [39]   
7                     [327, 23, 150, 349, 336, 6, 49]            [322]   
8   [128, 247, 44, 81, 272, 279, 282, 322, 331, 37...            [358]   
9          [95, 383, 262, 122, 44, 81, 252, 145, 336]            [440]   
10  [224, 251, 259, 485, 93, 437, 53, 166, 374, 44...            [467]   
11                      [471, 469, 355, 95, 243, 436]            [322]   
12        [374, 195, 109, 436, 95, 10,

In [115]:


f = open("C:/hlocal/TFM/vot_simple_waa", 'a')

# calculo la media de las metricas
for i in rango:
    name_one_hit = 'one_hit_' + str(i)
    name_mrr = 'mrr_' + str(i)
    name_precision = 'precision_' + str(i)
    name_recall = 'recall_' + str(i)
    name_f1 = 'f1_' + str(i)
    
    result_one_hit = metric_df[name_one_hit].mean()
    result_precision = metric_df[name_precision].mean()
    result_mrr = metric_df[name_mrr].mean()
    result_recall = metric_df[name_recall].mean()
    result_f1 = metric_df[name_f1].mean()

    # lo muestro por consola
    
    print(i)
    print("###########")
    print("One hit ----------")
    print(result_one_hit)
    print("Precision ----------")
    print(result_precision)
    print("Mrr  ----------")
    print(result_mrr)
    print("Recall  ----------")
    print(result_recall)
    print("F1  ----------")
    print(result_f1)
    
    
    f.write(str(result_one_hit) + '\t' + str(result_precision) + '\t' + str(result_mrr) + '\t' + str(result_recall) + '\t' +  str(result_f1) + '\n') 

    
f.close()    
    

1
###########
One hit ----------
0.06153846153846154
Precision ----------
0.06153846153846154
Mrr  ----------
0.06153846153846154
Recall  ----------
0.004510448567825617
F1  ----------
0.00810043846096977
2
###########
One hit ----------
0.09230769230769231
Precision ----------
0.05384615384615385
Mrr  ----------
0.07692307692307693
Recall  ----------
0.007443308058062157
F1  ----------
0.012434815277624642
3
###########
One hit ----------
0.1076923076923077
Precision ----------
0.041025641025641026
Mrr  ----------
0.08205128205128207
Recall  ----------
0.008404846519600618
F1  ----------
0.013150343808238544
4
###########
One hit ----------
0.12307692307692308
Precision ----------
0.038461538461538464
Mrr  ----------
0.08589743589743591
Recall  ----------
0.01084684896160306
F1  ----------
0.015903353057199213
5
###########
One hit ----------
0.16923076923076924
Precision ----------
0.04923076923076924
Mrr  ----------
0.09512820512820515
Recall  ----------
0.018459791574545675
F1  ---